In [ ]:
import gspread
import pandas as pd
import numpy as np

# Configure gspread

* Create a service account as described in the `gspread` documentation
* save the JSON key in this folder
* invite the relative `client_email` to be editor of the sheet

In [ ]:
import pathlib

In [ ]:
service_account_json = list(pathlib.Path(".").glob("*.json"))[0]

In [ ]:
print("Using credentials from", service_account_json)

In [ ]:
gc = gspread.service_account(filename=service_account_json)

In [ ]:
sh = gc.open("IRIS_telemetry_and_FITS_metadata")

print(sh.sheet1.get('B1'))

In [ ]:
worksheet = sh.get_worksheet(0)

In [ ]:
#worksheet.update([dataframe.columns.values.tolist()] + dataframe.values.tolist())

In [ ]:
isEvent_index = 13

In [ ]:
data = pd.read_csv("IRIS_telemetry_and_FITS_metadata.csv")

In [ ]:
missing_requiredrate = data[(data.isEvent == 1) & np.isnan(data.requiredRate)]

In [ ]:
row = missing_requiredrate.iloc[0]

In [ ]:
missing_requiredrate

In [ ]:
ls TCS-Model-Files/pk

In [ ]:
from pyhocon import ConfigFactory, ConfigMissingException



In [ ]:
for event in publish["publish"]["events"]:
    print(event["name"])

In [ ]:
row

In [ ]:
event

In [ ]:
row

In [ ]:
component_dict = {"TCS M3 Assembly":"m3a", "TCS M2 Assembly":"m2a", "cmIRIS":"iris", "ENCAssembly":"ecs/ENCAssemblyModelFiles",
                 "cmNFIRAOS":"nfiraos"}

In [ ]:
missing_rate = set()
for key, row in missing_requiredrate.iterrows():
    if row.subsystem == "TCS":
        component = component_dict.get(row.component, row.component)
        publish = ConfigFactory.parse_file(f'{row.subsystem}-Model-Files/{component}/publish-model.conf')
        event = [ev for ev in publish["publish"]["events"] if ev["name"] == row["name"]][0]

        variables = [at["name"] for at in event["attributes"]]

        try:
            assert row.Variable in variables
        except AssertionError:
            simplified_name = row.Variable
            for i in range(1, 10):
                simplified_name = simplified_name.replace(str(i), "")
            assert simplified_name in variables
            print(f"Using {simplified_name} for {row.Variable}")
 

        try:
            missing_requiredrate.at[key, "requiredRate"] = event["maxRate"]
        except ConfigMissingException:
            missing_rate.add(event["name"])

In [ ]:
event

In [ ]:
missing_requiredrate